In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime as dt
from pandas_datareader import DataReader as DR
import seaborn as sb
import numdifftools as nd
from wquantiles import quantile

from scipy.stats import multivariate_normal as multi_norm
from scipy.stats import multivariate_t as multi_t
from scipy.stats import norm,t,truncnorm
from scipy.spatial import Delaunay as TRI
from scipy.interpolate import LinearNDInterpolator as ITP
from scipy.optimize import minimize
from scipy.stats import gaussian_kde as KDE

from sklearn.linear_model import LinearRegression as Linear
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

import warnings
warnings.filterwarnings("ignore")

In [ ]:
class MLE:
    def __init__(self,dim,sigmaT,sigmaiP,df):
        self.dim=dim
        self.T=lambda x: norm.pdf(x=x,mean=np.zeros(dim),cov=sigmaT**2)
        self.T_1d=lambda x: norm.pdf(x=x,scale=sigmaT)
        self.iP=lambda x: t.pdf(x=x,df=k,loc=np.zeros(dim),shape=sigmaiP**2)
        self.iP_1d=lambda x: t.pdf(x=x,df=k,shape=sigmaT**2)
        self.iS=lambda size: t.rvs(size=size,df=k,loc=np.zeros(p),shape=sigmaT**2)
